# AITA CLASSIFCATION
This project aims to fine-tune a BERT language model to be able to accurately tell people on the r/amitheasshole (AITA) subreddit whether or not they are the asshole given the situation provided.
It is trained using posts scraped from the AITA subreddit, and the assigned labels to each prompt reflect the highest rated answer. Only highly upvoted examples appear in the dataset, as these will be the ones which face the most scrutiny and therefore will best reflect the overall sentiment. 

In [1]:
!pip install transformers
!pip install sentence-transformers
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm, trange
import torch
from torch.optim import AdamW
from torch import device
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import os
import pandas as pd
import numpy as np
import re 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import porter
import random
import json

print("IMPORTED LIBRARIES SUCCESSFULLY")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=3792f39da98b9cebde6f378b7131aca24e5a4af373e73cc464ffedcfc150916a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


IMPORTED LIBRARIES SUCCESSFULLY


In [2]:
try:
    from google.colab import drive
except:
    print("Could not import drive from google colab")

# Importing Data
Data is read from the JSON files containing the datasets, and labels are assigned based on what dataset the example comes in. Here, NTA (not the asshole) is labeled as 0, while YTA (you're the asshole) is labeled as 1. 

In [3]:
try:
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/AITA/data')
    os.getcwd()
except:
    pass
nta = pd.read_json("NTA_Dataset.jsonl", lines=True)
yta = pd.read_json("YTA_Dataset.jsonl", lines=True)
nta['label'] = 0
yta['label'] = 1

Mounted at /content/drive


# Pre-Processing
In this step, excess tags are removed, along with labels in the text that indicate whether or not the person is the asshole according to the response. 

In [4]:
def preprocess(data, column, remove_classification=False,set_to_lower=False,reduce_whitespace=False):
    # in: 2d table with prompts and resolution
    # out: tidied data
    
    # every line ends with 5 hashtags. This line removes all hashtags
    data[column] = data[column].str.replace(r'######', '', regex=True)

    if remove_classification:
      # removes the first occurence of 'NTA' or 'YTA' in the response
      data[column] = data[column].str.replace(r'[NnYy][Tt][Aa]', '', 1, regex=True)
    if set_to_lower:
      data[column] = data[column].str.lower()
    if reduce_whitespace:
      # replaces any whitespace characters with a single space - this includes new lines or multiple spaces
      data['completion'] = data['completion'].str.replace(r'\s+', ' ', regex=True)
    
    return data

In [5]:
def correct_labels(data):
  data.loc[(data["completion"].str.lower()).str.startswith(' definitely nta'), 'label'] = 0
  return data

In order to gather more data in the "YTA" category, the original data gatherers expanded their search to include responses starting with "Yes, YTA", "Yeah, YTA" and "Definitely YTA" (ignoring case and punctuation). However, some error must have come up in their original sorting, as 26 completions beginning with "Definitely NTA" are present in the YTA dataset. This code identifies these entries and corrects the labels. 

In [6]:
yta = correct_labels(yta)
data = pd.concat([nta,yta,yta])
data = preprocess(data,'completion')
data = preprocess(data,'prompt')
data

,prompt,completion,label
0,"I have a very decent name. I mean, I don't hat...",NTA you just pointed out a fact\n\nProtip :Yo...,0
1,**Not recent but we are still arguing over thi...,NTAIf she can't see other people drink how th...,0
2,I'm the eldest sibling in my family and have a...,NTA—your brother chose to isolate himself and...,0
3,My(28f) sister(20f) arrived in Canada for her ...,NTA—You’re only the asshole if you don’t tell...,0
4,So background I pay for Netflix and let my bro...,NTA—They abused your favor. They should get a...,0
...,...,...,...
2457,My daughter and her husband (34 and 33 respect...,YTA I hope this is fake because if not then ...,1
2458,My friend showed me her new website selling he...,"YTA for devaluing her work, her technical sk...",1
2459,So long story short my ex and I were living to...,YTA \n\n\nYou had already thrown away a bunc...,1
2460,"I have HSV-1, the virus that typically causes ...",YTA \n\n\nI didn't even need to read. You al...,1


The NTA datset is just over twice the size of the YTA dataset. In order to avoid a strong bias, we create our dataset with two copies of the YTA dataset and one of the NTA dataset.

# BERT CLASSIFICATION
Here, the response column is dropped from the dataset, as we already know whether or not the example is from the YTA or NTA dataset from the label, and the current task is binary classification.

In [7]:
data.drop(['completion'],axis=1,inplace=True)

In [8]:
t = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [9]:
max_len = 256
prompts = data.prompt.values
labes = data.label.values

In the following line, each prompt is encoded using the bert-base-uncased pretrained BERT tokenizer. Each prompt is converted into an array of numbers that represent the tokens. Additionally, special tokens are added in order to comply with the way that fine-tuning a BERT model operates, and sentences that are too long are truncated. The maximum length that a BERT model allows is 512, but we chose the max_len value of 256 as there were very few examples of sentences longer than 256 tokens.

Additionally, the sequences are padded so that every sequence of IDs is of the same length.
 
After this, a mask is applied to the sequence of IDs so that the transformer is able to pay attention to the actual IDs and ignore the padding when processing the data. 

In [10]:
prompt_ids = []
for i in trange(len(prompts)):
  prompt = prompts[i]
  e = t.encode(prompt, add_special_tokens = True, truncation=True, max_length=max_len)
  prompt_ids.append(e)
prompt_ids = pad_sequences(prompt_ids, maxlen=max_len, padding="post")

100%|██████████| 10238/10238 [01:24<00:00, 121.22it/s]


In [11]:
masks = []
for p in prompt_ids:
  masks.append([int(id > 0 ) for id in p])


# DATA PREPERATION
Here, the prompts, labels, and the prompt masks are converted into tensors.

Following this, the tensors, masks, and labels are compiled into a dataset, and this, along with a random sampler, composes as DataLoader, which allows for a pretrained model to be fine-tuned by traversing a dataset. 

In [12]:
tr_in, te_in, tr_lab, te_lab = train_test_split(prompt_ids, labes, test_size = .2)
tr_masks, te_masks, _, _ = train_test_split(masks, labes, test_size=.2)
tr_in = torch.tensor(tr_in)
te_in = torch.tensor(te_in)
tr_lab = torch.tensor(tr_lab)
te_lab = torch.tensor(te_lab)
tr_masks = torch.tensor(tr_masks)
te_masks = torch.tensor(te_masks)

In [13]:
batch_size = 16

In [14]:
tr_d = TensorDataset(tr_in, tr_masks, tr_lab)
tr_s = RandomSampler(tr_d)
tr_dl = DataLoader(tr_d, sampler = tr_s, batch_size = batch_size)
te_d = TensorDataset(te_in, te_masks, te_lab)
te_s = RandomSampler(te_d)
te_dl = DataLoader(te_d, sampler = te_s, batch_size = batch_size)

# MODEL SELECTION AND INITIALIZATION
Here, the sequence classification pretrained model from HuggingFace is employed, using "bert-base-uncased". 

The AdamW optimizer is also used, as this is the state of the art. Paired with this is a linear scheduler with warmup, which gradually changes the learning rate as the model progresses, allowing for it to generalize more and not get stuck during training on a local minima. 

In [15]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, output_attentions = False, output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
optimzier = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [17]:
epochs = 4
total_steps = len(tr_dl) * epochs
scheduler = get_linear_schedule_with_warmup(optimzier, num_warmup_steps = 0, num_training_steps = total_steps)

In [18]:
device = torch.device("cuda")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# MODEL TRAINING
The model training takes place over 4 epochs. Using the dataloader, tensors are sent to the GPU, and the model is evaluated based on its loss. This loss function is then backpropogated for weight updates, and the optimzer is also updated. The scheduler is updated after this, deciding on a new learning rate. 

After these 4 epochs are completed, the model is saved to the google drive. 

In [19]:
progress_bar = tqdm(range(total_steps))

model.train()
for epoch in range(4):
  print("Epoch " + str(epoch) +":")
  for step, batch in enumerate(tr_dl):
    b_id = batch[0].to(device)
    b_m = batch[1].to(device)
    b_lab = batch[2].to(device)
    outputs = model(b_id, token_type_ids=None, attention_mask=b_m, labels = b_lab)
    loss = outputs.loss
    loss.backward()

    optimzier.step()
    scheduler.step()
    optimzier.zero_grad()
    progress_bar.update(1)

  0%|          | 0/2048 [00:00<?, ?it/s]

Epoch 0:


  0%|          | 5/2048 [00:03<23:54,  1.42it/s]

KeyboardInterrupt: ignored

In [21]:
os.chdir('/content/drive/MyDrive/AITA/models')
model.save_pretrained(os.getcwd())
#chp = torch.load("pytorch_model.bin")
#model.load_state_dict(chp)


In [20]:
stops = stopwords.words("english")
lemmatizer = nltk.WordNetLemmatizer()
prompts = data.prompt.values
def process_prompt(sentence):
  # removes all preceding and final whitespace, all punctuation and special characters, and splits the sentence into a list of words
  sent = re.sub('[^\w\s]', '', sentence.lower().strip()).split()
  stemmer = porter.PorterStemmer()
  t_sent = [w for w in sent if w not in stops]
  t_sent = [stemmer.stem(w) for w in t_sent]
  t_sent = [lemmatizer.lemmatize(w) for w in t_sent]

  return " ".join(t_sent)

prompts_clean = [process_prompt(p) for p in prompts]

m = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = m.encode(prompts_clean[:20])
accusations_first_person = ["I was irresponsible","I was stupid","I was bigoted","I miscommunicated","My trauma caused this","I betrayed this person","I acted dangerously","I was obsessive","I was proud","I was insecure","I was rude"]
accusations_sec_person = ["You were irresponsible","You were stupid","You were bigoted","You miscommunicated","Your trauma caused this","You betrayed this person","You acted dangerously","You were obsessive","You were proud","You were insecure","You were rude"]
accusations_third_person = ["They were irresponsible","They were stupid","They were bigoted","They miscommunicated","Their trauma caused this","They betrayed you","They acted dangerously","They were obsessive","They were proud","They were insecure","They were rude"]
#neutral = ["The issue was caused by irresponsiblity","The issue was caused by entitlement","The issue was caused by stupidity","The issue was caused by bigotry","The issue was caused by miscommunication","The issue was caused by trauma","The issue was caused by betrayal","The issue was caused by unsafe actions","The issue was caused by obsession","The issue was caused by pride","The issue was caused by insecurity","The issue was caused by rudeness"]
categories = ["Irresponsiblity","Entitlement","Stupididty","Bigotry","Miscommunication","Trauma","Betrayal","Dangerousness","Obsession","Pride","Insecurity","Rudeness"]
cats_emb = m.encode(categories)
nta_acc_embedded = m.encode(accusations_third_person)
yta_acc_embedded = m.encode(accusations_first_person)

In [22]:
def test(sent):
  with torch.device("cuda") as device:
    predict_ids = []
    predict_mask = []
    predict_encoding = t.encode(sent, add_special_tokens = True, truncation=True, max_length=max_len)
    predict_ids.append(predict_encoding)
    predict_mask.append([int(id > 0 ) for id in predict_encoding])
    predict_ids = torch.tensor(predict_ids)
    predict_mask = torch.tensor(predict_mask)

    with torch.no_grad():
      output = model(predict_ids.to(device), token_type_ids = None, attention_mask = predict_mask)

    if np.argmax(output.logits.cpu()).flatten().item() == 1:
      return "YTA"
    else:
      return "NTA"

def calc_sliding_cosine_similarity(sent, embs, w):
  sims = []
  for i in range(len(sent)-w):
    sims.append(cosine_similarity([m.encode(sent[i:i+w])], embs))
  sims = np.array(sims).T
  sims = np.array([a[0] for a in sims])
  sims_avg = [np.mean(sim) for sim in sims]
  closest = np.argmax(sims_avg)
  return closest


def print_sample(n):
  for count, sentence in enumerate(prompts_clean[:n]):
    print(prompts[count])
    resp = test(prompts_clean[count])
    print("Response: " + resp)
    if resp == "NTA":
      reason = "The issue in this instance is their " +categories[calc_sliding_cosine_similarity(sentence, cats_emb, 15)]
    else:
      reason = "The issue in this instance is your " +categories[calc_sliding_cosine_similarity(sentence, cats_emb, 15)]
    print("Reason: " + reason)
    print("\n")
    print("**************************")
    print("\n")

In [ ]:
print_sample(20)

I have a very decent name. I mean, I don't hate it and I don't love it. Long story short, my father had a name picked out when I was born and up until birth, my name was going to be name A. Then, out came me and my mother freaked out, said she hated my name that they had agreed on for the past 7 months and said she was going to divorce my father if she every heard that name again. I ended up with name B, which has no relationship to name A at all. 

Now I have name B. Not a problem, but the thing of it is they haven't been able for the last 20+ years to agree on how to pronounce the damn thing. Think of it like LA Ah for Leah for one, and Leah for another. Or EE-Liz-abeth or Elizabeth. Two different, but very distinct sounding names, but it's the same name. I switch between the two, and prefer one over the other. 

My mother was bitching about my dad (they're divorced) over a mutual dilemma they had over my younger brother, and I finally said "I shouldn't be surprised you all can't dec

In [23]:
try:
  os.chdir('/content/drive/MyDrive/AITA/data')
  os.getcwd()
except:
  print("Failed to connect to drive.")
with open("tagged_data.json") as user_file:
    file_contents = json.load(user_file)
tagged = pd.DataFrame(file_contents)
tagged.drop(['completion'],axis=1,inplace=True)

In [ ]:
correct_resp = 0
correct_reason = 0

for index, row in tagged.iterrows():
  prompt = row['prompt']
  prompt = re.sub('[^\w\s]', '', prompt.lower().strip()).split()
  stemmer = porter.PorterStemmer()
  t_sent = [w for w in prompt if w not in stops]
  t_sent = [stemmer.stem(w) for w in t_sent]
  t_sent = [lemmatizer.lemmatize(w) for w in t_sent]
  prompt = " ".join(t_sent)
  resp = test(prompt)
  if (resp == "NTA") and (row['label'] == 0):
    correct_resp += 1
  elif resp == "YTA" and row['label'] == 1:
    correct_resp += 1
  reason = calc_sliding_cosine_similarity(prompt, cats_emb, 15)
  if reason == row['reason']:
    correct_reason += 1
correct_resp
correct_reason

28

In [ ]:
resp_success = correct_resp/200
reason_success = correct_reason/200
print("Response Accuracy:", resp_success)
print("Reason Accuracy:", reason_success)

Response Accuracy: 0.69
Reason Accuracy: 0.14
